In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (8, 8)
plt.rcParams["font.size"] = 14

# Regression and kernels

We now go back to regression problems, and to make things more exciting, we will work with real-world data! We are going to load the Boston dataset of house prices, and try to predict the value of a house based on its properties.

In [ ]:
from sklearn import datasets
dset = datasets.load_boston()
print(dset.DESCR)

As before, we randomly split the data in a training and test sets. We also apply some pre-processing routines, something fundamental when working with real data. Here we *standardize* each feature (i.e. make them zero-mean, unit-variance), and remove the mean of the target vector.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

X, y = dset.data, dset.target
print("number of samples/features: %d, %d" % X.shape)

# Split dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)
print("training/test samples: %d, %d" % (len(y_train), len(y_test)))

# Center and scale features and observations
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test = scaler.transform(X_test)
ymean = np.mean(y_train)
y_train = y_train - ymean
y_test = y_test - ymean

## Approximating kernels via random feature maps

Our next aim is to map the problem to a dimension $D>P$ (ideally, $D\gg P$). Our aim is to embed the original feature space in a higher dimensional space, in the hope that the problem can be treated by linear methods in the larger space. Then, linear maps are not very useful, because they would preserve the dimensionality of the original problem. So, we also have to use non-linear maps.

We thus define $z_i = f(x_i)$ using a non-linear map $f: \mathbb{R}^P \to \mathbb{R}^D$ with $D > P$. It was shown in the lectures that when $D\to\infty$, the scalar products $z_i^T z_j$ converge to a **kernel function** $K({x}_i, {x}_j)$.

For many interesting learning procedures, the embedding in higher dimension is therefore equivalent to the replacement of scalar products by a kernel. This essentially take us to infinite dimension!

Usually, kernels are normalized in such a way that $K(x,x)=1$.
In the following we focus more specifically on the RBF kernel $K({x}_i, {x}_j) = e^{-\frac{1}{2} \| {x}_i - {x}_j \|^2_2}$. 

**Exercise**: compute and plot the kernelized Gram matrix $K_{ij} = K(x_i,x_j)$ for our dataset.

In [ ]:
# Compute kernelized Gram matrix using the original kernel ...
from scipy.spatial.distance import cdist
gram_orig =

# ... and plot it.
plt.imshow(gram_orig, interpolation="nearest")

In [ ]:
# %load rks2.py

As we saw, it's often more efficient to *approximate* the kernel by expanding it in a given basis (usually Fourier) and then approximately performing the integral via importance sampling. This is equivalent to consider a *random* map (as we did in the dimensional reduction case, but now for a non-linear map)

$$\phi({x}_i, {w}) = e^{i w^T x_i}$$

with $w$ extracted i.i.d. from some $p(w)$ and 

$$z_i = f(x_i) = \frac1{\sqrt{D}}\{ \phi({x}_i, w_1),\cdots, \phi({x}_i, {w}_D) \}
 =\frac1{\sqrt{D}}\{ e^{i w_1^T x_i},\cdots, e^{i w_D^T x_i} \} \in \mathbb{C}^D
$$

Note that for convenience we used $\mathbb{C}^D$ as a feature space, which has dimension $2D$ instead of $D$.
Then

$$K({x}_i, {x}_j) = z_i^T z_j 
\approx \frac{1}{D} \sum_{a = 1}^D \phi({x}_i, {w}_a) \phi^*({x}_j, {w}_a)
\approx \int dw p(w) e^{i w^T (x_i - x_j)}
$$

If we choose $p(w) = e^{-\frac12 ||w||_2^2}/(2 \pi)^{P/2}$ as a Gaussian with unit variance, then we get the desired kernel.
This is the so-called *random kitchen sinks* algorithm. Let's implement it!

![RKS](rks.png)

In [ ]:
def generate_nonlinear_features(X, n_projections):
    n_features = X.shape[1]
    
    # Sample w
    w =
    
    # Compute z
    z =
    return z

In [ ]:
# %load rks3.py

We can see how the approximated Gram matrix approaches the exact one as `n_projections` increase.

In [ ]:
# Compute the Gram matrix using the approximate kernel
X_transf = generate_nonlinear_features(X_train, 500)
gram_approx = X_transf.dot(X_transf.T)

# Plot Gram matrices using both exact and approximate kernels
fig, axs = plt.subplots(1, 2, figsize = (12, 6))
p0 = axs[0].imshow(gram_orig, interpolation="nearest", vmin=0, vmax=0.1)
p1 = axs[1].imshow(gram_approx, interpolation="nearest", vmin=0, vmax=0.1)
plt.colorbar(p0, ax=axs[0], shrink=0.75)
plt.colorbar(p1, ax=axs[1], shrink=0.75)
plt.tight_layout()

In [ ]:
# Scatter plot of exact and approximate kernels
plt.plot(gram_orig.ravel(), gram_approx.ravel(), "o")
plt.plot([0, 1], [0, 1])

plt.xlabel("original kernel")
plt.ylabel("approximated kernel")

How many projections do we need before $\langle \phi (x) \phi(\mu)\rangle$ gives a good approximation to $K(x, \mu) = e^{-\frac{(x - \mu)^2}{2}}$?

In [ ]:
mu = 1.3

# Generate 1D grid
xx = np.linspace(-5, 5, 101)

# Compute kernel approximation
def compute_approx(grid, n_projections):
    phi = generate_nonlinear_features((grid - mu).reshape(-1, 1), n_projections)
    return np.sum(phi, 1) / np.sqrt(n_projections)

# Plot approximation and exact kernels
plt.plot(xx, compute_approx(xx, 100))
plt.plot(xx, np.exp(-.5 * (xx - mu) ** 2))

## Kernel ridge regression

Let's see how our kernel approximation performs in the solution of linear models. We first start with doing ridge regression while replacing the Gram matrix by the (exact) kernel.

This amounts to map $x_i$ into $z_i$, and then doing a ridge regression $y = Z w$ in the new feature space. The solution is 

$$w = Z^T (Z Z^T + \lambda I_N)^{-1} y = Z^T (K + \lambda I_N)^{-1} y$$

and if we are shown new data (e.g. the test set) we first compute $Z_{test}$ and then

$$y = Z_{test} w = Z_{test} Z^T (K + \lambda I_N)^{-1} y = K^{test}(K + \lambda I_N)^{-1} y$$

Therefore, we only need to compute the kernel $K_{ij} = K(x_i,x_j)$ and the test kernel $K^{test}_{ij} = K(x^{test}_i,x_j)$ to perform the ridge regression, without need to really compute the features $z_i$.

In [ ]:
def train_kernel_ridge(X, y, lamb=0.05):
    n_samples, n_features = X.shape
    
    # Obtain kernelized Gram matrix and perform least-squares estimation
    corr =
    coef =
    
    # Compute training error
    error = np.mean((y - corr.T.dot(coef)) ** 2)
    print("error on training set: %g" % error)
    
    return coef

def test_kernel_ridge(X_test, y_test, X_train, coef):
    corr =
    
    # Compute test (generalization) error
    error = np.mean((y_test - corr.T.dot(coef)) ** 2)
    print("error on test set: %g" % error)
    
    return error

coef = train_kernel_ridge(X_train, y_train)
kernel_ridge_error = test_kernel_ridge(X_test, y_test, X_train, coef)

In [ ]:
# %load rks4.py

Let us do a sanity check here, just to see if these numbers make sense. What happens if we try to do linear ridge regression?

In [ ]:
def train_ridge(X, y, lamb=0.05):
    n_samples, n_features = X.shape
    
    coef = np.linalg.lstsq(X.T.dot(X) + lamb * np.eye(n_features), X.T.dot(y))[0]
    error = np.mean((y_train - X_train.dot(coef)) ** 2)
    print("error on training set w/ ridge: %g" % error)
        
    return coef
    
coef_ridge = train_ridge(X_train, y_train)
ridge_test = np.mean((y_test - X_test.dot(coef_ridge)) ** 2)
print("error on test set w/ ridge: %g" % (ridge_test))

What we observe is that the kernel ridge regression is badly overfitting the train dataset, but yet it performs a bit better than the linear ridge regression on the test dataset.

Let us also check our results with the scikitlearn implementation.

In [ ]:
from sklearn import kernel_ridge

reg = kernel_ridge.KernelRidge(alpha=0.05, kernel="rbf", gamma=0.5)
reg.fit(X_train, y_train)
sklearn_train = np.mean((y_train - reg.predict(X_train)) ** 2)
sklearn_test = np.mean((y_test - reg.predict(X_test)) ** 2)
print("error on training/test set w/ sklearn: %g/%g" % (sklearn_train, sklearn_test))

Our results seem to match those of scikitlearn. To conclude, we can quickly check the behavior as a function of the regularization parameter $\lambda$.

In [ ]:
# Initialize empty arrays
lam_dom = np.logspace(-2, 2, 100)

train_err_k = np.zeros(len(lam_dom))
test_err_k = np.zeros(len(lam_dom))
train_err_nok = np.zeros(len(lam_dom))
test_err_nok = np.zeros(len(lam_dom))

n_samples, n_features = X_train.shape
train_corr = np.exp(-.5 * cdist(X_train, X_train) ** 2)
test_corr = np.exp(-.5 * cdist(X_train, X_test) ** 2)

# Compute different metrics for many $\lambda$
for (i, lamb) in enumerate(lam_dom):
    coef = np.linalg.lstsq(train_corr + lamb * np.eye(n_samples), y_train)[0]
    train_err_k[i] = np.mean((y_train - train_corr.T.dot(coef)) ** 2)
    test_err_k[i] = np.mean((y_test - test_corr.T.dot(coef)) ** 2)
    
    coef = np.linalg.lstsq(X_train.T.dot(X_train) + lamb * np.eye(n_features), X_train.T.dot(y_train))[0]
    train_err_nok[i] = np.mean((y_train - X_train.dot(coef)) ** 2)
    test_err_nok[i] = np.mean((y_test - X_test.dot(coef)) ** 2)
    
# Visualize
plt.semilogx(lam_dom,train_err_k, label='Kernel, train')
plt.semilogx(lam_dom,test_err_k, label='Kernel, test')
plt.semilogx(lam_dom,train_err_nok, label='No kernel, train')
plt.semilogx(lam_dom,test_err_nok, label='No kernel, test')
plt.legend()
plt.xlabel("$\lambda$")
plt.ylabel("error")

## Approximating the kernel

Now let's replace the exact kernel with the approximate one. We first rewrite the `generate_nonlinear_features` function so as to receive/return $\omega$ values; this is necessary in order to compute $K$ for new points outside the training set.

In [ ]:
def generate_nonlinear_features(X, n_projections, w = None):
    n_features = X.shape[1]
    if w is None:
        w = np.random.randn(n_features, n_projections)
    z = np.hstack((np.cos(X.dot(w)), np.sin(X.dot(w)))) / np.sqrt(n_projections)
    return z, w

We are now ready to rewrite the functions above using approximate kernels.

In [ ]:
def train_rks(X, y, n_projections, lamb=0.05):
    n_samples, n_features = X.shape
    # Obtain random features and perform least-squares estimation
    z, w = generate_nonlinear_features(X, n_projections)
    coef = z.T.dot(np.linalg.lstsq(z.dot(z.T) + lamb * np.eye(n_samples), y)[0])
    # Compute training error
    train_error = np.mean((y - z.dot(coef)) ** 2)
    return coef, w, train_error

def test_rks(X, y, coef, w):
    z, _ = generate_nonlinear_features(X, w.shape[1], w)
    # Compute test (generalization) error
    test_error = np.mean((y_test - z.dot(coef)) ** 2)
    return test_error

coef, w, train_error = train_rks(X_train, y_train, 500)
test_error = test_rks(X_test, y_test, coef, w)
print("error on train set: %g" % train_error)
print("error on test set: %g" % test_error)

What is the performance as a function of `n_projections`?

In [ ]:
# Compute test error for many values of n_projections
ks = np.arange(200, 5000, 200)
errors = np.zeros(len(ks))
for (i, k) in enumerate(ks):
    coef, w, _ = train_rks(X_train, y_train, k)
    errors[i] = test_rks(X_test, y_test, coef, w)

# Plot results
plt.plot(ks, errors)
plt.axhline(kernel_ridge_error, c="k", ls="-.", lw=3);

In the Boston dataset example we had $N\approx 500$ and $P=13$.
We found that with around $D\approx 1000$ projections, we can obtain roughly the same results that we obtained with the original kernel. Do we gain something using the projections?

To understand this, let us check the computational time for the different options:

$\bullet$ If we use the original kernel ridge regression, the kernel has size $N\times N$ and computing each element requires $P$ operations, therefore its computation takes time $\approx N^2 P$ and its inversion takes time $\approx N^3$. The total time is $N^2 \max(P, N)$.

$\bullet$ If we use $D$ random projections, then we need time $P D$ to construct the $w$, and time $N P D$ to compute the matrix $Z$. But then we have two options. If we use the formula

$$w_1 = Z^T (Z Z^T + \lambda I_N)^{-1} y$$

then we need to construct the matrix $Z Z^T$, which takes time $N^2 D$, and invert it, which takes time $N^3$. The total time is then $\max(N P D, N^2 D, N^3)$.
**But**, we can also use the equivalent formula

$$w_2 = (Z^T Z + \lambda I_D)^{-1} Z^T y$$

in which case we need to construct the matrix $Z^T Z$, which takes time $D^2 N$, and invert it, which takes time $D^3$. The total time is then $\max(N P D, D^2 N, D^3)$.



In the example above, we have small $P$, and we need $D>N$, hence the Kernel method takes time $\approx N^3$, while for the random projections method it is better to use $w_1$, which takes time $N^2 D$, so in the end the kernel method is faster. However, in a "big data" setting where $N \gg D \gg P$, then the kernel method would take time $N^3$, while the random projection method using $w_2$ would take time $N D^2$, providing a huge speedup.






## Kernel SVM

We didn't gain so much in using RKS in the previous section, partially because the dataset we were using was simple and small, as we discussed. Let us try it again with a bigger, more complex dataset.

Moreover, we will use Support Vector Machines. Recall that a SVM is a classifier, defined as follows. We first find the normal vector

$$\hat{w} = \text{argmin}_w  \frac1N \sum_{i=1}^N H[y_i (w \cdot x_i - b)] + \lambda ||w||^2  
\qquad \qquad H(z) = \max(0,1-z)$$

Then, the classifier for a new data $x$ is

$$y = \text{sgn}[ \hat{w} \cdot x - b ]$$

A kernel SVM consists, as before, in moving to a different feature space $z_i = f(x_i)$ and applying a linear SVM in the new feature space. Because it can be shown that $\hat w = \sum_{i=1}^N c_i y_i z_i$, one can express everything in terms of the kernel $K(x_i, x_j) = z_i \cdot z_j$. 

The training of a kernel SVM requires us to evaluate the kernel multiple times and not only once as in ridge regression. That makes even more important to optimize the computation.

We start by loading the MNIST dataset. We extract from it $N=5000$ samples (each with $P=784$ features), selected among the zeros and ones.

In [ ]:
from sklearn.datasets import fetch_mldata
mnist = fetch_mldata("MNIST original", data_home="../lec1")
X, y = mnist.data, mnist.target

# Preprocess
X = X / 255.

mask = np.logical_or.reduce([(y == k) for k in [0, 1]])
X, y = X[mask, :], y[mask]

n_samples = 5000
samples = np.random.choice(X.shape[0], n_samples, replace=False)
X, y = X[samples, :], y[samples]

# Do train/test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
print("samples/features in training set: %d, %d" % X_train.shape)

Then we run the kernel SVM function from scikit-learn. The fit time complexity is more than quadratic with the number of samples $N$.

In [ ]:
from time import time
from sklearn import svm

# Create kernel SVM, k(x) = exp(-gamma |x|^2)
kernel_svm = svm.SVC(gamma=.2)

# Fit kernel SVM
kernel_svm_time = time()
kernel_svm.fit(X_train, y_train)
kernel_svm_error = np.mean(y_test != kernel_svm.predict(X_test))
kernel_svm_time = time() - kernel_svm_time

print("error on training set: %g" % np.mean(y_train != kernel_svm.predict(X_train)))
print("error on test set: %g" % kernel_svm_error)
print("time: %g" % kernel_svm_time)

The kernel SVM provides a perfect fit of the training dataset, with a good error on the test dataset.

Finally, we repeat the process for the Fourier-approximated kernel SVM. We use the RBFSampler routine of sklearn, that approximates feature map of an RBF kernel by Monte Carlo approximation of its Fourier transform, via Random Kitchen Sinks.

In [ ]:
from sklearn import pipeline
from sklearn.kernel_approximation import RBFSampler

# Create Fourier-approximated SVM, k(x) = exp(-gamma |x|^2)
map_fourier = RBFSampler(gamma=.2)
fourier_svm = pipeline.Pipeline([("feature_map", map_fourier),
                                 ("svm", svm.LinearSVC())])

# Fit Fourier-approximated SVM
fourier_times = []
fourier_train_errors = []
fourier_test_errors = []

sample_sizes = [100, 200, 500, 1000, 2000, 5000, 10000]
for k in sample_sizes:
    fourier_svm.set_params(feature_map__n_components=k)
    start = time()
    fourier_svm.fit(X_train, y_train)
    fourier_times.append(time() - start)
    fourier_train_error = np.mean(y_train != fourier_svm.predict(X_train))
    fourier_train_errors.append(fourier_train_error)
    fourier_test_error = np.mean(y_test != fourier_svm.predict(X_test))
    fourier_test_errors.append(fourier_test_error)
        

Let us plot the results. We find that the fit time complexity is now linear in $D$, and we obtain an important speedup.

In [ ]:
# Plot results
fig, axs = plt.subplots(1, 2, figsize=(10,5))
axs[0].plot(sample_sizes, fourier_train_errors, label="Train")
axs[0].plot(sample_sizes, fourier_test_errors, label="Test")
axs[0].axhline(kernel_svm_error, c="k", ls="-.", lw=2)
axs[0].set(ylabel="error", xlabel="D")
axs[0].legend()
axs[1].plot(sample_sizes, fourier_times)
axs[1].axhline(kernel_svm_time, c="k", ls="-.", lw=2)
axs[1].set(ylabel="runtime", xlabel="D")
fig.tight_layout()